In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
import sys
import datetime

In [23]:


loader = TextLoader("llama2_text.txt")
documents=loader.load()

In [24]:
text_splitter=RecursiveCharacterTextSplitter(
                                             chunk_size=500,
                                             chunk_overlap=20)

In [25]:
text_chunks=text_splitter.split_documents(documents)

print(len(text_chunks))

34


In [26]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device':'cpu'})


#**Step 4: Convert the Text Chunks into Embeddings and Create a FAISS Vector Store***
vector_store=FAISS.from_documents(text_chunks, embeddings)


##**Step 5: Find the Top 3 Answers for the Query***

query="YOLOv7 outperforms which models"
docs = vector_store.similarity_search(query)

In [27]:
print(docs)

[Document(page_content="LLaMA (Large Language Model Meta AI) is a large language model (LLM) released by Meta AI in February 2023. Four model sizes were trained: 7, 13, 33 and 65 billion parameters. LLaMA's developers reported that the 13B parameter model's performance on most NLP benchmarks exceeded that of the much larger GPT-3 (with 175B parameters) and that the largest model was competitive with state of the art models such as PaLM and Chinchilla.[1] Whereas the most powerful LLMs have generally been accessible", metadata={'source': 'llama2_text.txt'}), Document(page_content='Vincent, James (8 March 2023). "Meta\'s powerful AI language model has leaked online â€” what happens now?". The Verge.\n "Meta and Microsoft Introduce the Next Generation of Llama". Meta. 18 July 2023. Retrieved 21 July 2023.\n Touvron, Hugo Touvron; Martin, Louis; et al. (18 Jul 2023). "Llama 2: Open Foundation and Fine-Tuned Chat Models". arXiv:2307.09288 [cs.CL].\n Shazeer, Noam (2020-02-01). "GLU Variants

In [28]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':500,
                          'temperature':0.1})

In [29]:
template="""Use the following pieces of information to answer the user's question.
If you dont know the answer just say you know, don't try to make up an answer.

Context:{context}
Question:{question}

Only return the helpful answer below and nothing else
Helpful answer
"""

qa_prompt=PromptTemplate(template=template, input_variables=['context', 'question'])

#start=timeit.default_timer()

chain = RetrievalQA.from_chain_type(llm=llm,
                                   chain_type='stuff',
                                   retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
                                   return_source_documents=True,
                                   chain_type_kwargs={'prompt': qa_prompt})

In [33]:
a = datetime.datetime.now()
question="when was llama announced?"
chat_history=[]
result=chain({'query':question,"chat_history": chat_history})
print(result['result'])
b = datetime.datetime.now()
print("time taken -",b-a)

LLaMA was announced on February 23, 2023 via a blog post and paper describing the model's training, architecture, and performance.
time taken - 0:01:03.556324


In [34]:
a = datetime.datetime.now()
question="What is length of  Llama 2 - Chat? "
chat_history=[]
result=chain({'query':question,"chat_history": chat_history})
print(result['result'])
b = datetime.datetime.now()
print("time taken -",b-a)

The length of Llama 2 - Chat is 4K tokens.
time taken - 0:00:56.699026
